**REFIT DATA** https://pure.strath.ac.uk/ws/portalfiles/portal/52873458/REFIT_Readme.txt

In [ ]:
import pandas as pd
import numpy as np
import pickle
import glob
from itertools import combinations_with_replacement, combinations
from scipy.stats import ks_2samp
from scipy.special import rel_entr

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, MeanShift, AffinityPropagation, SpectralClustering
from sklearn.metrics import mean_squared_error

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
new_path = "/Processed_Data_CSV.7z"

In [ ]:
#!pip install py7zr

In [ ]:
#import py7zr

In [ ]:
#with py7zr.SevenZipFile(new_path, mode='r') as z:
#    z.extractall(path="/")

## **Memory efficient way of importing data**

Import by chunks, skip every few rows </br>

In [ ]:
def hourly_data(df, time_column='Unix'):
  """
  This function returns the first reading of each hour
  """
  df_copy = df.copy()
  df_copy.sort_values(by='Unix', ascending=True, inplace=True)
  df_copy[time_column] = ONE_HOUR * (df_copy[time_column] // ONE_HOUR)
  df_copy.drop_duplicates(subset=time_column, keep='first', inplace=True)
  return df_copy

In [ ]:
def filter_sort_date(df_list, date_col_name='date'):
    """
    Filters dataframes of a list based on a roughly common date range
    """
    dates = [(min(df[date_col_name].tolist()), max(df[date_col_name].tolist())) for df in df_list]
    print([(i, d) for i, d in enumerate(dates)])
    min_date, max_date = max([x[0] for x in dates]), min([x[-1] for x in dates])
    #print(min_date, max_date)
    filtered_df_list = [df.loc[(df[date_col_name] >= min_date) & (df[date_col_name] <= max_date)].reset_index().sort_values(date_col_name) for df in df_list]
    return filtered_df_list

In [ ]:
real_col_names = {'1': ['Fridge', 'Chest Freezer', 'Upright Freezer', 'Tumble Dryer', 'Washing Machine', 'Dishwasher', 'Computer Site', 'Television Site', 'Electric Heater'], '2': ['Fridge-Freezer', 'Washing Machine', 'Dishwasher', 'Television', 'Microwave', 'Toaster', 'Hi-Fi', 'Kettle', 'Oven Extractor Fan'], '3': ['Toaster', 'Fridge-Freezer', 'Freezer', 'Tumble Dryer', 'Dishwasher', 'Washing Machine', 'Television', 'Microwave', 'Kettle'], '4': ['Fridge', 'Freezer', 'Fridge-Freezer', 'Washing Machine (1)', 'Washing Machine (2)', 'Computer Site', 'Television Site', 'Microwave', 'Kettle'], '5': ['Fridge-Freezer', 'Tumble Dryer 3', 'Washing Machine', 'Dishwasher', 'Computer Site', 'Television Site', 'Combination Microwave', 'Kettle', 'Toaster'], '6': ['Freezer (Utility Room)', 'Washing Machine', 'Dishwasher', 'MJY Computer', 'Television Site', 'Microwave', 'Kettle', 'Toaster', 'PGM Computer'], '7': ['Fridge', 'Freezer (Garage)', 'Freezer', 'Tumble Dryer', 'Washing Machine', 'Dishwasher', 'Television Site', 'Toaster', 'Kettle'], '8': ['Fridge', 'Freezer', 'Dryer', 'Washing Machine', 'Toaster', 'Computer', 'Television Site', 'Microwave', 'Kettle'], '9': ['Fridge-Freezer', 'Washer Dryer', 'Washing Machine', 'Dishwasher', 'Television Site', 'Microwave', 'Kettle', 'Hi-Fi', 'Electric Heater'], '10': ['Magimix (Blender)', 'Freezer', 'Chest Freezer (In Garage)', 'Fridge-Freezer', 'Washing Machine', 'Dishwasher', 'Television Site', 'Microwave', 'Kenwood KMix'], '11': ['Fridge', 'Fridge-Freezer', 'Washing Machine', 'Dishwasher', 'Computer Site', 'Microwave', 'Kettle', 'Router', 'Hi-Fi'], '12': ['Fridge-Freezer', 'Television Site(Lounge)', 'Microwave', 'Kettle', 'Toaster', 'Television Site(Bedroom)', 'Not Used', 'Not Used', 'Not Used'], '13': ['Television Site', 'Unknown', 'Washing Machine', 'Dishwasher', 'Tumble Dryer', 'Television Site', 'Computer Site', 'Microwave', 'Kettle'], '15': ['Fridge-Freezer', 'Tumble Dryer', 'Washing Machine', 'Dishwasher', 'Computer Site', 'Television Site', 'Microwave', 'Kettle ', 'Toaster'], '16': ['Fridge-Freezer (1)', 'Fridge-Freezer (2)', 'Electric Heater (1)', 'Electric Heater (2)', 'Washing Machine', 'Dishwasher', 'Computer Site', 'Television Site', 'Dehumidifier/Heater'], '17': ['Freezer (Garage)', 'Fridge-Freezer', 'Tumble Dryer (Garage)', 'Washing Machine', 'Computer Site', 'Television Site', 'Microwave', 'Kettle', 'Plug Site (Bedroom)'], '18': ['Fridge(garage)', 'Freezer(garage)', 'Fridge-Freezer', 'Washer Dryer(garage)', 'Washing Machine', 'Dishwasher', 'Desktop Computer', 'Television Site', 'Microwave'], '19': ['Fridge & Freezer', 'Washing Machine', 'Television Site', 'Microwave', 'Kettle', 'Toaster', 'Bread-maker', 'Lamp (80Watts)', 'Hi-Fi'], '20': ['Fridge', 'Freezer', 'Tumble Dryer', 'Washing Machine', 'Dishwasher', 'Computer Site', 'Television Site', 'Microwave', 'Kettle'], '21': ['Fridge-Freezer', 'Tumble Dryer', 'Washing Machine', 'Dishwasher', 'Food Mixer', 'Television', 'Kettle/Toaster', 'Vivarium', 'Pond Pump']}

In [ ]:
categories_refit = {'fridge': ['Fridge', 'Freezer', 'Chest Freezer', 'Upright Freezer', 'Fridge-Freezer', 'Freezer (Utility Room)', 'Freezer (Garage)', 'Chest Freezer (In Garage)', 'Fridge-Freezer (1)', 'Fridge-Freezer (2)', 'Fridge(garage)', 'Freezer(garage)', 'Fridge & Freezer'], 'washing_drying': ['Tumble Dryer', 'Washing Machine', 'Dishwasher', 'Washing Machine (1)', 'Washing Machine (2)', 'Dryer', 'Tumble Dryer (Garage)', 'Washer Dryer(garage)', 'Washer Dryer'], 'computer_tv': ['Computer Site', 'MJY Computer', 'PGM Computer,Computer', 'Desktop Computer'] + ['Television Site', 'Television', 'Television Site(Lounge)', 'Television Site(Bedroom)'], 'heating': ['Electric Heater', 'Microwave', 'Toaster', 'Kettle', 'Combination Microwave', 'Electric Heater', 'Electric Heater (2)', 'Dehumidifier/Heater', 'Kettle/Toaster'], 'other': ['Hi-Fi', 'Oven Extractor Fan', 'Magimix (Blender)', 'Kenwood KMix', 'Router', 'Plug Site (Bedroom)', 'Lamp (80Watts)', 'Bread-maker', 'Food Mixer', 'Vivarium', 'Pond Pump']}

**Filter to show the first reading of every hour**</br>
**If I want to sum for each hour, I'll just sum later on. But that might be unfair for different readings**

In [ ]:
ONE_HOUR = 3600

def harmonize_timestamps(df_list, time_column='Unix'):
  """
  This function filters datasets to keep hours in common
  """
  hourly_list = [df.set_index(time_column, drop=False).drop('index', axis=1) for df in df_list]
  timestamps_list = [set(df[time_column].tolist()) for df in hourly_list]
  common_hours = sorted(set.intersection(*timestamps_list))
  valid_df_list = [df.loc[common_hours] for df in hourly_list]
  del hourly_list

  return valid_df_list

In [ ]:
def create_category_cols(df, categories, remove_other=False):
  df_copy = df.copy()
  cols = df_copy.columns.tolist()
  cat_cols = {c: [col for col in cols if col in categories[c]] for c in categories.keys()}

  for cat_col in cat_cols:
    cols_in_df = [x for x in categories[cat_col] if x in cols]
    if cols_in_df:
      df_copy[cat_col] = df_copy[cols_in_df].sum(axis=1).values

    df_copy.drop(cols_in_df, axis=1, inplace=True)

  for col in df_copy.columns:
    if col not in ["Unix", "Aggregate"] + list(categories.keys()):
      try:
        df_copy.drop(col, axis=1, inplace=True)
      except KeyError:
        continue

  if remove_other and 'other' in df_copy.columns:
    df_copy.drop('other', axis=1, inplace=True)
  return df_copy

In [ ]:
all_files_new

['/content/drive/MyDrive/Energy_Data/House_3.csv',
 '/content/drive/MyDrive/Energy_Data/House_1.csv',
 '/content/drive/MyDrive/Energy_Data/House_10.csv',
 '/content/drive/MyDrive/Energy_Data/House_4.csv',
 '/content/drive/MyDrive/Energy_Data/House_11.csv',
 '/content/drive/MyDrive/Energy_Data/House_5.csv',
 '/content/drive/MyDrive/Energy_Data/House_12.csv',
 '/content/drive/MyDrive/Energy_Data/House_6.csv',
 '/content/drive/MyDrive/Energy_Data/House_13.csv',
 '/content/drive/MyDrive/Energy_Data/House_15.csv',
 '/content/drive/MyDrive/Energy_Data/House_7.csv',
 '/content/drive/MyDrive/Energy_Data/House_16.csv',
 '/content/drive/MyDrive/Energy_Data/House_8.csv',
 '/content/drive/MyDrive/Energy_Data/House_17.csv',
 '/content/drive/MyDrive/Energy_Data/House_9.csv',
 '/content/drive/MyDrive/Energy_Data/House_18.csv',
 '/content/drive/MyDrive/Energy_Data/House_19.csv',
 '/content/drive/MyDrive/Energy_Data/House_2.csv',
 '/content/drive/MyDrive/Energy_Data/House_20.csv',
 '/content/drive/MyDr

**Import data**

In [ ]:
df_list_new = []
house_nbs = []
all_files_new = glob.glob("/content/drive/MyDrive/Energy_Data" + "/*.csv")
cols__ = ["Unix", "Aggregate", "Appliance1", "Appliance2", "Appliance3", "Appliance4", "Appliance5", "Appliance6", "Appliance7", "Appliance8", "Appliance9"]

step_size = 1
for ind, f in enumerate(all_files_new):
  house_nb = int(f[f.index('House_')+6:f.index('.csv')])
  house_nbs.append(house_nb)

  current_df = pd.read_csv(f, usecols=cols__)
  current_df.columns = ["Unix", "Aggregate"] + real_col_names[str(house_nb)]
  current_df = hourly_data(current_df)

  print('dataframe size is ', len(current_df))
  df_list_new.append(current_df.iloc[::step_size])
  del current_df

#df_list_new = [df_list_new[i] for i in house_nbs]

#df_new = pd.concat(df_list_new,ignore_index=True)

dataframe size is  13132
dataframe size is  13520
dataframe size is  12949
dataframe size is  13584
dataframe size is  8417
dataframe size is  14535
dataframe size is  10985
dataframe size is  11829
dataframe size is  9436
dataframe size is  12130
dataframe size is  12929
dataframe size is  11155
dataframe size is  12003
dataframe size is  10632
dataframe size is  11581
dataframe size is  10042
dataframe size is  10644
dataframe size is  11354
dataframe size is  10412
dataframe size is  10701


In [ ]:
for i, h in enumerate(df_list_new):
  print(i, create_category_cols(h, categories_refit, remove_other=True).columns.tolist())

print('\n Remove houses with less categories, the 6th and 8th house')

0 ['Unix', 'Aggregate', 'fridge', 'washing_drying', 'computer_tv', 'heating']
1 ['Unix', 'Aggregate', 'fridge', 'washing_drying', 'computer_tv', 'heating']
2 ['Unix', 'Aggregate', 'fridge', 'washing_drying', 'computer_tv', 'heating']
3 ['Unix', 'Aggregate', 'fridge', 'washing_drying', 'computer_tv', 'heating']
4 ['Unix', 'Aggregate', 'fridge', 'washing_drying', 'computer_tv', 'heating']
5 ['Unix', 'Aggregate', 'fridge', 'washing_drying', 'computer_tv', 'heating']
6 ['Unix', 'Aggregate', 'fridge', 'computer_tv', 'heating']
7 ['Unix', 'Aggregate', 'fridge', 'washing_drying', 'computer_tv', 'heating']
8 ['Unix', 'Aggregate', 'washing_drying', 'computer_tv', 'heating']
9 ['Unix', 'Aggregate', 'fridge', 'washing_drying', 'computer_tv', 'heating']
10 ['Unix', 'Aggregate', 'fridge', 'washing_drying', 'computer_tv', 'heating']
11 ['Unix', 'Aggregate', 'fridge', 'washing_drying', 'computer_tv', 'heating']
12 ['Unix', 'Aggregate', 'fridge', 'washing_drying', 'computer_tv', 'heating']
13 ['Unix',

In [ ]:
df_list_new = [create_category_cols(h, categories_refit, remove_other=True) for h in df_list_new]
df_list_new = [h for i, h in enumerate(df_list_new) if i not in ([6, 8])]

house_nbs = [h for i, h in enumerate(house_nbs) if i not in ([6, 8])]

In [ ]:
filt = filter_sort_date(df_list_new, date_col_name='Unix')
del df_list_new # to ease on the memory

[(0, (1380135600, 1433239200)), (1, (1381323600, 1436526000)), (2, (1384945200, 1435662000)), (3, (1381485600, 1436259600)), (4, (1401793200, 1435676400)), (5, (1380186000, 1436202000)), (6, (1385640000, 1435528800)), (7, (1387299600, 1436320800)), (8, (1383343200, 1436320800)), (9, (1389348000, 1436320800)), (10, (1383343200, 1431298800)), (11, (1394121600, 1434722400)), (12, (1387299600, 1436378400)), (13, (1394186400, 1432465200)), (14, (1394121600, 1434769200)), (15, (1379455200, 1432800000)), (16, (1395316800, 1435086000)), (17, (1394208000, 1436526000))]


In [ ]:
tt = [(0, (1395316800, 1435086000)), (1, (1394208000, 1436526000)), (2, (1380135600, 1433239200)), (3, (1381323600, 1436526000)), (4, (1384945200, 1435662000)), (5, (1381485600, 1436259600)), (6, (1401793200, 1435676400))]

In [ ]:
filt = harmonize_timestamps(filt)

In [ ]:
filt[0].head()

,Unix,Aggregate,fridge,washing_drying,computer_tv,heating
Unix,,,,,,
1401793200,1401793200,4418,193,2128,0,0
1401796800,1401796800,2446,86,0,143,0
1401800400,1401800400,590,250,0,143,0
1401804000,1401804000,346,193,0,0,0
1401807600,1401807600,213,0,0,0,0


### **Comparisons**

**Using clustering directly**

In [ ]:
to_comp = pd.concat([x['Aggregate'] for x in filt], axis=1, ignore_index=True).dropna().T
to_comp.head()

Unix,1401793200,1401796800,1401800400,1401804000,1401807600,1401811200,1401814800,1401818400,1401822000,1401825600,...,1431266400,1431270000,1431273600,1431277200,1431280800,1431284400,1431288000,1431291600,1431295200,1431298800
0,4418,2446,590,346,213,592,592,592,592,592,...,2594,443,186,478,434,380,505,465,458,402
1,211,151,223,221,175,149,151,311,287,487,...,274,324,280,285,243,257,182,3424,406,426
2,1120,367,623,605,479,1030,609,881,537,491,...,2921,663,274,253,261,303,251,3433,2969,277
3,400,268,516,534,458,404,902,168,551,434,...,164,275,174,267,369,210,402,276,526,275
4,1777,1351,2302,920,1039,431,1874,149,247,61,...,702,1698,428,245,448,368,137,66,403,358


In [ ]:
model = KMeans(n_clusters=2)
model.fit(to_comp)
yhat = model.predict(to_comp)

clusters = np.unique(yhat)
print('K-means', yhat)

K-means [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


**Overall comparison using mean**

In [ ]:
means = [d.mean() for d in filt]

In [ ]:
means

In [ ]:
fig, ax = plt.subplots(nrows=(len(filt[0].columns[2:])//3)+1, ncols=3, figsize=(12, 10))
fig.tight_layout()

reg_df = []
for i, col in enumerate(filt[0].columns[1:]):
  values_to_comp = [d.loc[col] for d in means]
  reg_df.append(values_to_comp)
  ax[i//3, i%3].scatter(range(len(means)), values_to_comp)
  ax[i//3, i%3].set_title(col)

  for label, x, y in zip(['house_'+str(x) for x in range(len(means))], range(len(means)), values_to_comp):
    ax[i//3, i%3].annotate(label,xy=(x, y), xytext=(-20, 20),
        textcoords='offset points', ha='left', va='bottom',
        bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
        arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))

In [ ]:
pd.DataFrame(reg_df).T

**Cluster using overall consumption (no temporal patterns) and all features**

In [ ]:
model = KMeans(n_clusters=2)
model.fit(pd.DataFrame(reg_df).T)
yhat_kmeans = model.predict(pd.DataFrame(reg_df).T)

clusters = np.unique(yhat_kmeans)
print('K-means', yhat_kmeans)

K-means [0 1 0 1 1 0 1 1 0 0 0 1 0 1 1 1 1 0]


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


**Cluster using overall consumption|occupancy|owned appliances** 


This gave the same result as using **overall consumption** only.

In [ ]:
advanced_df = pd.concat([pd.DataFrame(reg_df).T, pd.DataFrame({'occupancy': [2, 4, 2, 2, 4, 2, 4, 2, 2, 4, 1, 1, 6, 3, 2, 4, 2, 4], 'owned_appliances': [35, 15, 27, 33, 44, 49, 25, 35, 24, 31, 25, 19, 48, 22, 34, 26, 39, 23]})], axis=1)

In [ ]:
advanced_model = KMeans(n_clusters=2)
advanced_model.fit(advanced_df.iloc[:,[0,-1,-2]].values)
advanced_yhat_kmeans = advanced_model.predict(advanced_df.iloc[:,[0,-1,-2]].values)

clusters = np.unique(advanced_yhat_kmeans)
print('advanced_K-means', advanced_yhat_kmeans)

advanced_K-means [1 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1]


/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## **Package new data as usable data**

In [ ]:
# Output data format 
"""
train is a list of weekly data (here maybe not weekly) je ne sais pas, I'll start with chunks
train[week] = {
    'X': {appliance_category: appliance_df_for_week}
    'X_bar': x_bar_df_for_week
}
appliance_df_for_week: 
    - each row has the hourly consumption for the appliance_category (first reading or sum)
    - each column corresponds to a house

x_bar_df_for_week:
    - each row has the aggregate hourly consumption
    - each column corresponds to a house
"""

"\ntrain is a list of weekly data (here maybe not weekly) je ne sais pas, I'll start with chunks\ntrain[week] = {\n    'X': {appliance_category: appliance_df_for_week}\n    'X_bar': x_bar_df_for_week\n}\nappliance_df_for_week: \n    - each row has the hourly consumption for the appliance_category (first reading or sum)\n    - each column corresponds to a house\n\nx_bar_df_for_week:\n    - each row has the aggregate hourly consumption\n    - each column corresponds to a house\n"

In [ ]:
yhat_kmeans

array([1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1], dtype=int32)

In [ ]:
source_ind, target_ind = np.where(yhat_kmeans==0)[0], np.where(yhat_kmeans==1)[0]

In [ ]:
source_ind, target_ind

array([ 0,  3,  5,  6,  8,  9, 13, 15, 16, 17])

In [ ]:
TEST_SIZE = int(len(target_ind)*0.5)
np.random.choice(target_ind, TEST_SIZE, replace=False)

array([ 9, 16, 13,  3,  8])

In [ ]:
# train test split

test_prop = 0.5

TEST_SIZE = int(len(target_ind)*0.5)

train_ind = list(source_ind) + list(np.random.choice(target_ind, TEST_SIZE, replace=False))
target_starts_at = len(list(source_ind))

test_ind = [ind for ind in target_ind if not ind in train_ind]

train_houses = [house for i, house in enumerate(filt) if i in train_ind]
test_houses = [house for i, house in enumerate(filt) if i in test_ind]

In [ ]:
filt[0].head()

,Unix,Aggregate,fridge,washing_drying,computer_tv,heating
Unix,,,,,,
1401793200,1401793200,365,196,0,4,2
1401796800,1401796800,219,116,0,4,2
1401800400,1401800400,206,85,0,82,2
1401804000,1401804000,261,109,0,4,2
1401807600,1401807600,230,125,0,4,2


In [ ]:
target_starts_at

8

In [ ]:
def per_appliance_data(df_list, appliance, inds, target_starts_at=None):
  per_app_data = [x[appliance] for x in df_list]

  if not target_starts_at:
    cols = ['house_'+str(house_nbs[i]) for i in inds]
  else:
    source_cols = ['house_'+str(house_nbs[i]) for i in train_ind[:target_starts_at]]
    target_cols = ['house_'+str(house_nbs[i])+'_target' for i in train_ind[target_starts_at:]]
    cols = source_cols + target_cols

  per_appliance_df = pd.concat(per_app_data, axis=1)
  per_appliance_df.columns = cols
  return per_appliance_df

In [ ]:
CHUNK_SIZE = 24 * 7 # roughly weekly but not quite...
def chunk_data(df, chunk_size=CHUNK_SIZE):
  chunk_list = []
  length = len(df)
  current_ind = 0
  while current_ind < length:
    chunk_list.append(df.iloc[current_ind: current_ind + chunk_size])
    current_ind += chunk_size
  return chunk_list

In [ ]:
test_houses

In [ ]:
chunked_train = [chunk_data(house) for house in train_houses]
chunked_train = list(map(list, zip(*chunked_train)))

chunked_test = [chunk_data(house) for house in test_houses]
chunked_test = list(map(list, zip(*chunked_test))) #transpose so that the 1st dim is chunks, the 2nd dim is houses


In [ ]:
cols_to_use = ['Appliance1', 'Appliance2', 'Appliance3', 'Appliance4', 'Appliance5', 'Appliance6', 'Appliance7', 'Appliance8', 'Appliance9']
new_cols_to_use = ['fridge', 'washing_drying', 'computer_tv', 'heating'] #remove 'Unix', 'Aggregate', 
agg_col = 'Aggregate'

In [ ]:
def create_final_data(chunk_list, ind, cols_to_use, target_starts_at=None):
  final_list = []
  for chunk in range(len(chunk_list)):
    final_list.append({
        'X': {k: per_appliance_data(chunk_list[chunk], k, ind, target_starts_at) for k in cols_to_use},
        'X_bar': per_appliance_data(chunk_list[chunk], agg_col, ind, target_starts_at)
    })
  return final_list


In [2]:
final_train_data = create_final_data(chunked_train, train_ind, new_cols_to_use, target_starts_at)
final_test_data = create_final_data(chunked_test, test_ind, new_cols_to_use)

In [ ]:
with open('/content/drive/MyDrive/refit_data_by_category.pk', 'wb') as output_file:
  pickle.dump({
      'train': final_train_data,
      'test': final_test_data
  }, output_file)